# DDQN

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from torch.optim.lr_scheduler import CosineAnnealingLR
from collections import deque, namedtuple


class DDQN(nn.Module):

  def __init__(self, state_size, action_size):
    super(DDQN, self).__init__()
    # self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)
  

# Defining the hyperparameters

# learning_rate = 5e-4 # 范围[0.1, 0.0001], 学习率越大, 学习速度越快, 但可能会导致不稳定
# minibatch_size = 100 # 从经验回放中抽取的样本数
# discount_factor = 0.99 # 折扣因子, 用于计算Q值, 越大越重视未来奖励
# replay_buffer_size = int(1e5) # 经验回放缓冲区大小, 用于存储之前的经验
# interpolation_parameter = 1e-3 # 插值参数, 用于更新目标网络的参数, 即更新速度, 越小越稳定, 越大越快. 范围[0, 1]

# Episode 100	Average Score: -142.85
# Episode 200	Average Score: -113.38
# Episode 300	Average Score: -75.679
# Episode 400	Average Score: -8.597
# Episode 500	Average Score: 167.36
# Episode 600	Average Score: 205.04
# Episode 700	Average Score: 230.95
# Episode 800	Average Score: 204.73
# Episode 900	Average Score: 227.66
# Episode 1000	Average Score: 241.06
# Episode 1100	Average Score: 233.71
# Episode 1200	Average Score: 223.04
# Episode 1300	Average Score: 239.03
# Episode 1346	Average Score: 250.07
# Environment solved in 1246 episodes!	Average Score: 250.07

learning_rate = 0.01
minibatch_size = 100 # 从经验回放中抽取的样本数
discount_factor = 0.995 # 折扣因子, 用于计算Q值, 越大越重视未来奖励
replay_buffer_size = int(1e6) # 经验回放缓冲区大小, 用于存储之前的经验
interpolation_parameter = 0.001 # 插值参数, 用于更新目标网络的参数, 即更新速度, 越小越稳定, 越大越快. 范围[0, 1]


# Implementing the Experience Replay

class Memory(object):

  def __init__(self, batch_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.batch_size = batch_size
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.batch_size:
      del self.memory[0]

  # def sample(self, batch_size):
  #   experiences = random.sample(self.memory, k = batch_size)
  #   states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
  #   actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
  #   rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
  #   next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
  #   dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
  #   return states, next_states, actions, rewards, dones
  
  def prepare_indices(self):
    # Calculate rewards for each experience
    rewards = np.array([e[2] for e in self.memory if e is not None])
    # Sort indices by reward size
    sorted_indices = np.argsort(rewards)

    # Divide samples into higher and lower reward groups
    self.high_reward_indices = sorted_indices[-len(sorted_indices) // 2:]
    self.low_reward_indices = sorted_indices[:len(sorted_indices) // 2]

  def sample(self, batch_size):
    selected_high = np.random.choice(self.high_reward_indices, batch_size // 2, replace=False)
    selected_low = np.random.choice(self.low_reward_indices, batch_size // 2, replace=False)

    # Combine selected indices
    selected_indices = np.concatenate([selected_high, selected_low])

    # Retrieve experiences using selected indices
    experiences = [self.memory[i] for i in selected_indices]
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)

    return states, next_states, actions, rewards, dones


class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.episode_counter = 0  # New counter for episodes
    self.state_size = state_size
    self.action_size = action_size
    self.local_model = DDQN(state_size, action_size).to(self.device) # 本地网络, 用于学习, 参数不断更新. 用于选择动作
    self.target_model = DDQN(state_size, action_size).to(self.device) # 目标网络, 用于预测, 参数固定不变. 用于计算Q值
    self.optimizer = optim.Adam(self.local_model.parameters(), lr = learning_rate)
    # self.scheduler = StepLR(self.optimizer, step_size = 10, gamma = 0.9999) # 学习率调度器, 每隔20个epoch, 学习率乘以0.95
    self.scheduler = CosineAnnealingLR(self.optimizer, T_max = 50, eta_min = 0.001) # T_max: 重启周期, eta_min: 学习率下限
    self.memory = Memory(replay_buffer_size)
    self.t_step = 0 # 计数器, 用于更新目标网络, 每隔一定步数更新一次
    self.loss = 0

  # def step(self, state, action, reward, next_state, done):
  #   self.memory.push((state, action, reward, next_state, done))
  #   self.t_step = (self.t_step + 1) % 4 # 每隔4步更新目标网络一次
  #   if self.t_step == 0:
  #     if len(self.memory.memory) > minibatch_size: # 当经验回放中的样本数大于minibatch_size时, 从中抽取样本, 否则不更新网络
  #       experiences = self.memory.sample(minibatch_size)
  #       self.learn(experiences, discount_factor)
        
  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    if (self.episode_counter + 1) % 10 == 0:  # Every 10 episodes, prepare indices
        self.memory.prepare_indices()
    self.episode_counter += 1  # Increment episode count

    # Check if it's time to learn
    if len(self.memory.memory) > minibatch_size:
      experiences = self.memory.sample(minibatch_size)
      self.learn(experiences, discount_factor)
    

  # 根据epsilon贪心策略选择动作
  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_model.eval() # 将网络设置为评估模式 (需要根据观察到的状态选择动作, 为了该过程的可靠性, 模型需要一个非学习的稳定状态)
    with torch.no_grad(): # 不进行梯度计算
      action_values = self.local_model(state)
    self.local_model.train() # 设置回训练模式
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  # 从记忆库中抽取样本, 计算Q值, 更新网络参数
  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_model(next_states).detach().max(1)[0].unsqueeze(1) # 用目标网络, 预测下一个状态的Q值
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones) 
    q_expected = self.local_model(states).gather(1, actions) # 用本地网络, 预测当前状态的Q值
    loss = F.mse_loss(q_expected, q_targets) # 计算损失, 即Q值的差异
    self.loss = loss.item() 
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.scheduler.step()
    self.soft_update(self.local_model, self.target_model, interpolation_parameter)

  # 通过local network的参数更新target network的参数
  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

  
  def get_loss(self):
    return self.loss


# Setting up the Lunar Environment

import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

### Initialize the Agent

agent = Agent(state_size, number_actions)


### Training the agent

number_episodes = 1000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.990
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100) # 保存最近100个episode的分数


with open("rewards11.csv", 'w') as f:
  f.write("Episode,Reward,Loss,Done\n")
for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  loss = 0

  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    loss = agent.get_loss()
    state = next_state
    score += reward
    if done:
      break
      
  with open("rewards.csv", "a") as f:
    f.write(str(episode) + ',' + str(score) + ',' + str(loss)+ ',' + str(done) + '\n')
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_model.state_dict(), f'checkpoint{episode}.pth')
  # if np.mean(scores_on_100_episodes) >= 250.0:
  #   print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
  #   torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
  #   break



# DQN

In [ ]:
# DQN
# import random
import numpy as np
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
import gymnasium as gym
from collections import deque
from torch.optim.lr_scheduler import StepLR
from torch import nn

env = gym.make("LunarLander-v2")


class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = DQN(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.scheduler = StepLR(self.optimizer, step_size=100, gamma=0.95)

    def act(self, state):
        if isinstance(state, np.ndarray):
            state = torch.from_numpy(state).float().to(self.device)
        elif isinstance(state, torch.Tensor) and state.device != self.device:
            state = state.to(self.device)

        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)

        self.model.eval()
        with torch.no_grad():
            action_values = self.model(state.unsqueeze(0))
        self.model.train()

        return action_values.argmax().item()

    def store(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        
        minibatch = random.sample(self.memory, batch_size)
        states, actions, rewards, next_states, dones = zip(*minibatch)

        states = torch.tensor(states, dtype=torch.float32).to(self.device)
        actions = torch.tensor(actions, dtype=torch.long).to(self.device).unsqueeze(1)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(self.device).unsqueeze(1)
        next_states = torch.tensor(next_states, dtype=torch.float32).to(self.device)
        dones = torch.tensor(dones, dtype=torch.bool).to(self.device).unsqueeze(1)
        
        rewards *= rewards

        self.model.eval()
        next_q_values = self.model(next_states).detach().max(1)[0].unsqueeze(1)
        self.model.train()
        q_targets = rewards + self.gamma * next_q_values * (~dones)
        q_expected = self.model(states).gather(1, actions)

        loss = F.mse_loss(q_expected, q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            self.scheduler.step()
            
import os
import os
from torch.optim.lr_scheduler import StepLR



# 设置环境和代理
env = gym.make("LunarLander-v2")
agent = DQNAgent(env.observation_space.shape[0], env.action_space.n)

# 检查和设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
agent.model.to(device)  # 确保模型在正确的设备上

# 创建模型保存目录
current_dir = os.getcwd()
model_dir = os.path.join(current_dir, "models")
os.makedirs(model_dir, exist_ok=True)

# 训练参数
e_range = 10000
time_range = 500
batch_size = 64

for e in range(e_range):
    full_state = env.reset()
    if isinstance(full_state, tuple):  # 检查是否是元组
        state = full_state[0]  # 假设实际状态在元组的第一位置
    else:
        state = full_state

    state = torch.from_numpy(state).float().to(device)  # 现在确保它是NumPy数组后转换为适合模型的张量形式，并直接移动到GPU

    cumulative_reward = 0

    for time in range(time_range):
        action = agent.act(state)
        next_state, reward, done, truncated, info = env.step(action)
        cumulative_reward += reward
        
        if done:
            agent.store(state.cpu().numpy(), action, reward, next_state, done) 

        state = torch.from_numpy(np.array(next_state)).float().to(device)

        if done:
            print(f"Episode: {e}/{e_range}, time: {time}, terminated: {done}, truncated: {truncated}, cumulative reward: {cumulative_reward}")
            break

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    
    agent.scheduler.step()  # 更新学习率调度器
    
    if e % 100 == 0:
        model_path = os.path.join(model_dir, f"model_0.pth")
        torch.save(agent.model.state_dict(), model_path)
        print("模型状态字典已保存至", model_path)

# PPO

In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
import torch
import os
import csv

current_dir = os.path.dirname(__file__)
models_path = os.path.join(current_dir, "models")
if not os.path.exists(models_path):
    os.makedirs(models_path)

env = gym.make('LunarLander-v2')
env.reset()

model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=f"{current_dir}/tensorboard")
# model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=17000, reset_num_timesteps=False)
model.save(f"{models_path}/ppo_lunar")